# 02 Ranked Interpretations

This notebook defines both algorithms to rank the valuations of a knowledgebase and the statements in them.

In [1]:
import os
import sys
sys.path.append(os.path.join('..', 'common'))

from datatypes import KnowledgeBase, Valuation, Alphabet, Normally, Formula, Literal, Atom, Bot, Top
from util import print_knowledge_base, all_valuations, print_valuation, entails, materialized

from frozendict import frozendict

from typing import Mapping, Set, Optional
from numbers import Number

In [2]:
f = Literal(Atom('f')) # flies
b = Literal(Atom('b')) # is a bird
p = Literal(Atom('p')) # is a pengiun
w = Literal(Atom('w')) # has wings
bot = Bot()            # Falsum
top = Top()            # Verum
K = {-(p >> b) / Bot(), b / f, p / -f, b / w}
alph = {lit.atom for lit in (p, b, f, w)}
print_knowledge_base(K)

{ p |~ ¬f, b |~ f, b |~ w, p → b }


First we define the ranking of the models/interpretations. That is a mapping from the valuation to a number $i \in \mathbb{N} \cup \{\infty\}$.

In [3]:
RankedModel = Mapping[Valuation, Number]

In [4]:
def valuation_in_min(antecedent: Formula, valuation: Valuation, ranked_model: RankedModel) -> bool:
    i = ranked_model[valuation]
    for valuation, rank in ranked_model.items():
        if rank >= i:
            continue
        if antecedent(valuation):
            return False
    return True

In [5]:
def violates_statement(statement: Normally, valuation: Valuation, ranked_model: RankedModel):
    antecedent = statement.left
    if not valuation_in_min(antecedent, valuation, ranked_model):
        return False
    if statement.materialize()(valuation):
        return False
    return True

The algorithm is defined like in the slides (albeit that this is probably not the most efficient implementation).

In [6]:
def minimal_ranked_model(knowledge_base: KnowledgeBase, alphabet: Optional[Alphabet] = None):
    if alphabet is None:
        alphabet = {atom for statement in knowledge_base for atom in statement.atoms}
    ranked_model = {}
    U = set(frozendict(valuation) for valuation in all_valuations(alphabet, True))
    for valuation in U:
        ranked_model[valuation] = 0
    i = 0
    V = U
    V_ = set()
    while True:
        V_ = V
        V = {valuation for valuation in U if ranked_model[valuation] == i and any(
            violates_statement(statement, valuation, ranked_model) for statement in knowledge_base)}
        if V == V_:
            for valuation in V:
                ranked_model[valuation] = float('inf')
            break  # Done
        else:
            i = i + 1
            for valuation in V:
                ranked_model[valuation] = i
    return ranked_model

In [7]:
def print_ranked_model(ranked_model: RankedModel, alphabet: Optional[Alphabet] = None):
    r = max(rank for rank in ranked_model.values() if rank < float('inf'))
    pad = len(str(r)) + 1
    if float('inf') in ranked_model.values():
        print('∞:'.rjust(pad), '-' * 8)
        for valuation, rank in ranked_model.items():
            if rank == float('inf'):
                print(' ' * (pad + 2), end='')
                print_valuation(valuation, alphabet)
    if len(ranked_model) > 1:
        for i in reversed(range(r + 1)):
            print('{}:'.format(i).rjust(pad), '-' * 8)
            for valuation, rank in ranked_model.items():
                if rank == i:
                    print(' ' * (pad + 2), end='')
                    print_valuation(valuation, alphabet)


In [8]:
print_knowledge_base(K)
min_rank_model = minimal_ranked_model(K)
print_ranked_model(min_rank_model, alph)

{ p |~ ¬f, b |~ f, b |~ w, p → b }
∞: --------
    { p w     } { f b     }
    { p       } { f w b   }
    { p f w   } { b       }
    { p f     } { b w     }
2: --------
    { p f b   } { w       }
    { p f b w } {         }
1: --------
    { f b     } { p w     }
    { b w     } { p f     }
    { p b w   } { f       }
    { b       } { p f w   }
    { p b     } { f w     }
0: --------
    { w       } { p f b   }
    {         } { p b f w }
    { f w     } { p b     }
    { f b w   } { p       }
    { f       } { p w b   }


The second approach is to rank the statements directly. Again the algorithm of the slides is used.

In [9]:
StatementRanking = Mapping[Number, Set[Normally]]

def statement_ranking(knowledge_base: KnowledgeBase) -> StatementRanking:
    E = {0: knowledge_base}
    rank = {}
    i = 0
    while True:
        E[i + 1] = set()
        for statement in E[i]:
            if entails(materialized(E[i]), -statement.left):
                E[i + 1].add(statement)
        if E[i] == E[i + 1]:
            break
        rank[i] = E[i] - E[i + 1]
        i = i + 1
    rank[float('inf')] = E[i]
    return rank


In [10]:
def print_statement_ranking(statement_ranking: StatementRanking):
    r = len(statement_ranking) - (float('inf') in statement_ranking)
    pad = len(str(r)) + 1

    if len(statement_ranking) > 1:
        for i in range(r):
            print('{}:'.format(i).rjust(pad), end=' ')
            statements = statement_ranking[i]
            print_knowledge_base(statements)

    if float('inf') in statement_ranking.keys():
        print('∞:'.rjust(pad), end=' ')
        statements = statement_ranking[float('inf')]
        print_knowledge_base(statements)


In [11]:
print_knowledge_base(K)
rank = statement_ranking(K)
print_statement_ranking(rank)

{ p |~ ¬f, b |~ f, b |~ w, p → b }
0: { b |~ f, b |~ w }
1: { p |~ ¬f }
∞: { p → b }
